<a href="https://colab.research.google.com/github/ladychavella/Implementation-YOLOv7-Detection-Riot-and-Fire-at-Music-Concerts/blob/main/IMPLEMENTASI_YOLOv7_UNTUK_DETEKSI_KERUSUHAN_KONSER_MUSIK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DETEKSI KERUSUHAN KONSER MUSIK DENGAN YOLOv7**


# Setup Lingkungan

## Instalasi

In [ ]:
!pip install numpy==1.23.5 --force-reinstall
!pip uninstall -y tensorflow jax jaxlib jax[cpu]
!pip install tensorboard torch-tb-profiler
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 28.1 MB/s eta 0:00:00


## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Path kerja ke direktori
%cd /content/drive/MyDrive/Skripsi_YOLOv7

ValueError: mount failed

## Clone YOLOv7

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

# Load Dataset

## Download Dataset Roboflow

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="0yN9LC6mgjiKuYv767b8")
project = rf.workspace("skripsi-umyk5").project("kerusuhan-konser-musik")
version = project.version(10)
dataset = version.download("yolov7")


## Distribusi Dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from collections import Counter
import seaborn as sns
import pandas as pd

# Lokasi folder dataset
label_dir = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/train/labels'
image_dir = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/train/images'


# Inisialisasi list label
labels = []

# Loop semua file label
for file_name in os.listdir(label_dir):
    with open(os.path.join(label_dir, file_name), 'r') as f:
        for line in f.readlines():
            class_id = int(line.split()[0])
            labels.append(class_id)

# Mapping label
label_map = {
    0: 'aman',
    1: 'kerusuhan_api',
    2: 'kerusuhan_massa'
}

# Konversi ID ke nama kelas
label_names = [label_map[label] for label in labels]

# Hitung distribusi label
label_counts = Counter(label_names)
df = pd.DataFrame.from_dict(label_counts, orient='index', columns=['Jumlah Gambar'])
df = df.reset_index().rename(columns={'index': 'Kelas'})

# Plot distribusi kelas
plt.figure(figsize=(8, 5))
sns.barplot(x='Kelas', y='Jumlah Gambar', data=df, palette='viridis')
plt.title('Distribusi Gambar per Kelas')
plt.xlabel('Kelas')
plt.ylabel('Jumlah')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

## Jumlah bounding box perkelas

In [ ]:
#Jumlah bounding box perkelas dari semua file labels

import os
from collections import defaultdict

# Path ke folder label
label_dirs = [
    "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/train/labels",
    "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/valid/labels",
    "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"
]

total_boxes = 0
class_counts = defaultdict(int)  # Menyimpan jumlah per class ID

# Proses pembacaan semua file label
for label_dir in label_dirs:
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r") as f:
                lines = f.readlines()
                total_boxes += len(lines)
                for line in lines:
                    class_id = int(line.strip().split()[0])
                    class_counts[class_id] += 1

# Output total
print("Total bounding box dalam dataset:", total_boxes)

# Output per kelas
print("\nJumlah bounding box per class ID:")
for class_id, count in sorted(class_counts.items()):
    print(f"Class {class_id}: {count} bounding box")


# Training Model

## training awal

In [ ]:
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

!wandb disabled
!python train.py \
    --workers 2 \
    --img 640 \
    --batch 16 \
    --epochs 100 \
    --data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
    --cfg cfg/training/yolov7.yaml \
    --weights 'yolov7.pt' \
    --hyp data/hyp.scratch.p5.yaml \
    --device 0


## training lanjutan

In [ ]:
!python train.py \
    --workers 2 \
    --img 640 \
    --batch 16 \
    --epochs 100 \
    --data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
    --cfg cfg/training/yolov7.yaml \
    --weights '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt' \
    --hyp data/hyp.scratch.p5.yaml \
    --device 0 \
    --name exp_continue

## Visualisasi Loss dan mAP

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Ganti path ini ke lokasi sebenarnya dari results.png kamu
results_img_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/results.png'

# Baca dan tampilkan gambar
img = mpimg.imread(results_img_path)
plt.figure(figsize=(12, 6))
plt.imshow(img)
plt.axis('off')  # Hilangkan axis biar bersih
plt.title('Hasil Training YOLOv7')
plt.show()


menampilkan result training lanjutan

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Path hasil training lanjutan
results_img_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/results.png'

# Baca dan tampilkan gambar
img = mpimg.imread(results_img_path)
plt.figure(figsize=(12, 6))
plt.imshow(img)
plt.axis('off')  # Hilangkan sumbu
plt.title('Hasil Training Lanjutan YOLOv7')
plt.show()


visualisasi map metrik

In [ ]:
import matplotlib.pyplot as plt

# GANTI path ini ke folder training lanjutan kamu
results_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/results.txt'

# Inisialisasi list untuk tiap metrik
train_loss, val_loss, map50, map95 = [], [], [], []

# Baca isi file dan ambil metriknya
with open(results_path, 'r') as f:
    lines = f.readlines()

for line in lines:
    parts = line.strip().split()
    try:
        train_loss.append(float(parts[2]))  # Train obj_loss
        val_loss.append(float(parts[3]))    # Val obj_loss
        map50.append(float(parts[8]))       # mAP@.5
        map95.append(float(parts[9]))       # mAP@.5:.95
    except:
        continue

# Plot metrik
epochs = range(1, len(train_loss)+1)
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Val Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, map50, label='mAP@0.5')
plt.plot(epochs, map95, label='mAP@0.5:0.95')
plt.title('mAP Score')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluasi otomatis
if len(train_loss) < 10:
    print("Data terlalu sedikit untuk analisis otomatis.")
else:
    last_train_loss = train_loss[-1]
    last_val_loss = val_loss[-1]
    delta = abs(last_train_loss - last_val_loss)

    if delta > 0.01 and last_val_loss > last_train_loss:
        print("Terindikasi overfitting: train loss jauh lebih kecil dari val loss.")
    elif last_train_loss > 0.07 and max(map50) < 0.3:
        print("Terindikasi underfitting: train loss masih tinggi dan mAP rendah.")
    elif max(map50) > 0.5:
        print("Model terlihat cukup baik, mAP tinggi.")
    else:
        print("Model belum optimal tapi tidak terlalu buruk. Coba tambahkan data atau atur hyperparameter.")

# Ambil nilai mAP tertinggi
best_map50 = max(map50)
best_map95 = max(map95)
epoch_best_map = map50.index(best_map50) + 1  # karena index mulai dari 0

print(f"\nModel mencapai nilai mAP@0.5 terbaik sebesar {best_map50:.3f} pada epoch ke-{epoch_best_map}.")
print(f"Sedangkan nilai mAP@0.5:0.95 terbaik yang diperoleh adalah {best_map95:.3f}.")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Ganti sesuai path runs/ kamu
base_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue'

# Daftar file metrik
curve_files = {
    'F1 Curve': 'F1_curve.png',
    'Precision-Recall Curve': 'PR_curve.png',
    'Precision Curve': 'P_curve.png',
    'Recall Curve': 'R_curve.png'
}

# Tampilkan semua gambar
plt.figure(figsize=(16, 8))
for i, (title, filename) in enumerate(curve_files.items(), 1):
    img_path = os.path.join(base_path, filename)
    if os.path.exists(img_path):
        img = mpimg.imread(img_path)
        plt.subplot(2, 2, i)
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
    else:
        print(f"File tidak ditemukan: {filename}")

plt.tight_layout()
plt.show()


# Testing Model

In [ ]:
#optional kondisional
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

!wget https://raw.githubusercontent.com/WongKinYiu/yolov7/main/detect.py -O detect.py


testing dengan model ke 1 conf 0.2

In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import glob

# Masuk ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

# Folder input dan output
test_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images'
output_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results'
weights_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt'

# Hapus hasil sebelumnya
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Jalankan deteksi
!python detect.py \
    --weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt \
    --conf 0.2 \
    --iou-thres 0.3 \
    --img-size 640 \
    --source /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images \
    --save-txt \
    --project /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results \
    --name deteksi_folder \
    --exist-ok \


# Tampilkan semua hasil deteksi dalam grid
result_folder = os.path.join(output_folder, 'deteksi_folder')
detected_images = sorted(glob.glob(os.path.join(result_folder, '*.jpg')))

print(f"\nTotal hasil deteksi: {len(detected_images)} gambar\n")
print("PATH hasil deteksi:", result_folder)
print("Gambar yang ditemukan:", detected_images)


# Grid config
cols = 1  # jumlah kolom dalam grid
rows = (len(detected_images) + cols - 1) // cols  # jumlah baris otomatis
plt.figure(figsize=(20, 8 * rows))  # ukuran figure per baris

for i, img_path in enumerate(detected_images):
    img = Image.open(img_path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{i+1}. {os.path.basename(img_path)}", fontsize=10)

plt.tight_layout()
plt.show()


tabel model 1 conf 0.2

In [ ]:
import os
from collections import Counter
import pandas as pd

# Mapping class ID ke nama
class_map = {
    0: 'aman',
    1: 'kerusuhan_api',
    2: 'kerusuhan_massa'
}

# Lokasi folder prediksi dan ground truth
label_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels"
ground_truth_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"  # Ganti sesuai folder GT kamu

# Fungsi hitung jumlah deteksi
def count_detections(label_folder):
    counts = Counter()
    for file in os.listdir(label_folder):
        if file.endswith('.txt'):
            with open(os.path.join(label_folder, file), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls_id = int(line.strip().split()[0])
                    counts[class_map.get(cls_id, 'unknown')] += 1
    return counts

# Fungsi hitung akurasi sederhana
def compute_accuracy(pred_folder, gt_folder):
    total = 0
    correct = 0
    for file in os.listdir(gt_folder):
        if file.endswith('.txt'):
            gt_path = os.path.join(gt_folder, file)
            pred_path = os.path.join(pred_folder, file)
            if not os.path.exists(pred_path):
                continue
            with open(gt_path, 'r') as f1, open(pred_path, 'r') as f2:
                gt_lines = [line.strip().split()[0] for line in f1]
                pred_lines = [line.strip().split()[0] for line in f2]
                total += len(gt_lines)
                correct += sum(1 for g, p in zip(gt_lines, pred_lines) if g == p)
    return round(correct / total * 100, 2) if total > 0 else None

# Hitung deteksi & akurasi
counts = count_detections(label_folder)
accuracy = compute_accuracy(label_folder, ground_truth_folder)

# Buat dataframe hasil
result = {
    'Model': 'yolov7',
    'Conf': 0.2,  # Ganti sesuai nilai deteksi kamu
    'IoU': 0.3,   # Ganti sesuai nilai deteksi kamu
    'Kerusuhan Massa': counts.get('kerusuhan_massa', 0),
    'Kerusuhan Api': counts.get('kerusuhan_api', 0),
    'Aman': counts.get('aman', 0),
    'Total Deteksi': sum(counts.values()),
    'Akurasi (%)': accuracy
}

df = pd.DataFrame([result])
print(df)

# Simpan ke CSV
df.to_csv("/content/hasil_perbandingan_MODEL_1_conf02.csv", index=False)
print("Hasil disimpan ke: hasil_perbandingan_model.csv")


testing dengan model ke 2 conf 0.2

In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import glob

# Masuk ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

# Folder input dan output
test_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images'
output_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results'
weights_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt'

# Hapus hasil sebelumnya
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Jalankan deteksi
!python detect.py \
    --weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt \
    --conf 0.2 \
    --iou-thres 0.3 \
    --img-size 640 \
    --source /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images \
    --save-txt \
    --project /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results \
    --name deteksi_folder \
    --exist-ok \


# Tampilkan semua hasil deteksi dalam grid
result_folder = os.path.join(output_folder, 'deteksi_folder')
detected_images = sorted(glob.glob(os.path.join(result_folder, '*.jpg')))

print(f"\nTotal hasil deteksi: {len(detected_images)} gambar\n")
print("PATH hasil deteksi:", result_folder)
print("Gambar yang ditemukan:", detected_images)


# Grid config
cols = 1  # jumlah kolom dalam grid
rows = (len(detected_images) + cols - 1) // cols  # jumlah baris otomatis
plt.figure(figsize=(20, 8 * rows))  # ukuran figure per baris

for i, img_path in enumerate(detected_images):
    img = Image.open(img_path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{i+1}. {os.path.basename(img_path)}", fontsize=10)

plt.tight_layout()
plt.show()


Tabel model ke 2 conf 0.2

In [ ]:
import os
from collections import Counter
import pandas as pd

# Mapping class ID ke nama
class_map = {
    0: 'aman',
    1: 'kerusuhan_api',
    2: 'kerusuhan_massa'
}

# Lokasi folder prediksi dan ground truth
label_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels"
ground_truth_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"  # Ganti sesuai folder GT kamu

# Fungsi hitung jumlah deteksi
def count_detections(label_folder):
    counts = Counter()
    for file in os.listdir(label_folder):
        if file.endswith('.txt'):
            with open(os.path.join(label_folder, file), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls_id = int(line.strip().split()[0])
                    counts[class_map.get(cls_id, 'unknown')] += 1
    return counts

# Fungsi hitung akurasi sederhana
def compute_accuracy(pred_folder, gt_folder):
    total = 0
    correct = 0
    for file in os.listdir(gt_folder):
        if file.endswith('.txt'):
            gt_path = os.path.join(gt_folder, file)
            pred_path = os.path.join(pred_folder, file)
            if not os.path.exists(pred_path):
                continue
            with open(gt_path, 'r') as f1, open(pred_path, 'r') as f2:
                gt_lines = [line.strip().split()[0] for line in f1]
                pred_lines = [line.strip().split()[0] for line in f2]
                total += len(gt_lines)
                correct += sum(1 for g, p in zip(gt_lines, pred_lines) if g == p)
    return round(correct / total * 100, 2) if total > 0 else None

# Hitung deteksi & akurasi
counts = count_detections(label_folder)
accuracy = compute_accuracy(label_folder, ground_truth_folder)

# Buat dataframe hasil
result = {
    'Model': 'yolov7',
    'Conf': 0.2,  # Ganti sesuai nilai deteksi kamu
    'IoU': 0.3,   # Ganti sesuai nilai deteksi kamu
    'Kerusuhan Massa': counts.get('kerusuhan_massa', 0),
    'Kerusuhan Api': counts.get('kerusuhan_api', 0),
    'Aman': counts.get('aman', 0),
    'Total Deteksi': sum(counts.values()),
    'Akurasi (%)': accuracy
}

df = pd.DataFrame([result])
print(df)

# Simpan ke CSV
df.to_csv("/content/hasil_perbandingan_MODEL_2_CONFT02.csv", index=False)
print("Hasil disimpan ke: hasil_perbandingan_model.csv")


testing dengan model ke 2 ukuran conf 0.25

In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import glob

# Masuk ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

# Folder input dan output
test_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images'
output_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results'
weights_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt'

# Hapus hasil sebelumnya
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Jalankan deteksi
!python detect.py \
    --weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt \
    --conf 0.25 \
    --iou-thres 0.45 \
    --img-size 640 \
    --source /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images \
    --save-txt \
    --project /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results \
    --name deteksi_folder \
    --exist-ok \


# Tampilkan semua hasil deteksi dalam grid
result_folder = os.path.join(output_folder, 'deteksi_folder')
detected_images = sorted(glob.glob(os.path.join(result_folder, '*.jpg')))

print(f"\nTotal hasil deteksi: {len(detected_images)} gambar\n")
print("PATH hasil deteksi:", result_folder)
print("Gambar yang ditemukan:", detected_images)


# Grid config
cols = 1  # jumlah kolom dalam grid
rows = (len(detected_images) + cols - 1) // cols  # jumlah baris otomatis
plt.figure(figsize=(20, 8 * rows))  # ukuran figure per baris

for i, img_path in enumerate(detected_images):
    img = Image.open(img_path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{i+1}. {os.path.basename(img_path)}", fontsize=10)

plt.tight_layout()
plt.show()


Tabel model ke 2 conf 0.25

In [ ]:
import os
from collections import Counter
import pandas as pd

# Mapping class ID ke nama
class_map = {
    0: 'aman',
    1: 'kerusuhan_api',
    2: 'kerusuhan_massa'
}

# Lokasi folder prediksi dan ground truth
label_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels"
ground_truth_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"  # Ganti sesuai folder GT kamu

# Fungsi hitung jumlah deteksi
def count_detections(label_folder):
    counts = Counter()
    for file in os.listdir(label_folder):
        if file.endswith('.txt'):
            with open(os.path.join(label_folder, file), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls_id = int(line.strip().split()[0])
                    counts[class_map.get(cls_id, 'unknown')] += 1
    return counts

# Fungsi hitung akurasi sederhana
def compute_accuracy(pred_folder, gt_folder):
    total = 0
    correct = 0
    for file in os.listdir(gt_folder):
        if file.endswith('.txt'):
            gt_path = os.path.join(gt_folder, file)
            pred_path = os.path.join(pred_folder, file)
            if not os.path.exists(pred_path):
                continue
            with open(gt_path, 'r') as f1, open(pred_path, 'r') as f2:
                gt_lines = [line.strip().split()[0] for line in f1]
                pred_lines = [line.strip().split()[0] for line in f2]
                total += len(gt_lines)
                correct += sum(1 for g, p in zip(gt_lines, pred_lines) if g == p)
    return round(correct / total * 100, 2) if total > 0 else None

# Hitung deteksi & akurasi
counts = count_detections(label_folder)
accuracy = compute_accuracy(label_folder, ground_truth_folder)

# Buat dataframe hasil
result = {
    'Model': 'yolov7',
    'Conf': 0.25,  # Ganti sesuai nilai deteksi kamu
    'IoU': 0.45,   # Ganti sesuai nilai deteksi kamu
    'Kerusuhan Massa': counts.get('kerusuhan_massa', 0),
    'Kerusuhan Api': counts.get('kerusuhan_api', 0),
    'Aman': counts.get('aman', 0),
    'Total Deteksi': sum(counts.values()),
    'Akurasi (%)': accuracy
}

df = pd.DataFrame([result])
print(df)

# Simpan ke CSV
df.to_csv("/content/hasil_perbandingan_MODEL_2_CONF_025.csv", index=False)
print("Hasil disimpan ke: hasil_perbandingan_model.csv")


Testing dengan model ke 1 ukuran conf 0.25

In [ ]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import glob

# Masuk ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7

# Folder input dan output
test_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images'
output_folder = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results'
weights_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt'

# Hapus hasil sebelumnya
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Jalankan deteksi
!python detect.py \
    --weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt \
    --conf 0.25 \
    --iou-thres 0.45 \
    --img-size 640 \
    --source /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/images \
    --save-txt \
    --project /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results \
    --name deteksi_folder \
    --exist-ok \


# Tampilkan semua hasil deteksi dalam grid
result_folder = os.path.join(output_folder, 'deteksi_folder')
detected_images = sorted(glob.glob(os.path.join(result_folder, '*.jpg')))

print(f"\nTotal hasil deteksi: {len(detected_images)} gambar\n")
print("PATH hasil deteksi:", result_folder)
print("Gambar yang ditemukan:", detected_images)


# Grid config
cols = 1  # jumlah kolom dalam grid
rows = (len(detected_images) + cols - 1) // cols  # jumlah baris otomatis
plt.figure(figsize=(20, 8 * rows))  # ukuran figure per baris

for i, img_path in enumerate(detected_images):
    img = Image.open(img_path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{i+1}. {os.path.basename(img_path)}", fontsize=10)

plt.tight_layout()
plt.show()


Tabel model ke 1 conf 0.25

In [ ]:
import os
from collections import Counter
import pandas as pd

# Mapping class ID ke nama
class_map = {
    0: 'aman',
    1: 'kerusuhan_api',
    2: 'kerusuhan_massa'
}

# Lokasi folder prediksi dan ground truth
label_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels"
ground_truth_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"  # Ganti sesuai folder GT kamu

# Fungsi hitung jumlah deteksi
def count_detections(label_folder):
    counts = Counter()
    for file in os.listdir(label_folder):
        if file.endswith('.txt'):
            with open(os.path.join(label_folder, file), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls_id = int(line.strip().split()[0])
                    counts[class_map.get(cls_id, 'unknown')] += 1
    return counts

# Fungsi hitung akurasi sederhana
def compute_accuracy(pred_folder, gt_folder):
    total = 0
    correct = 0
    for file in os.listdir(gt_folder):
        if file.endswith('.txt'):
            gt_path = os.path.join(gt_folder, file)
            pred_path = os.path.join(pred_folder, file)
            if not os.path.exists(pred_path):
                continue
            with open(gt_path, 'r') as f1, open(pred_path, 'r') as f2:
                gt_lines = [line.strip().split()[0] for line in f1]
                pred_lines = [line.strip().split()[0] for line in f2]
                total += len(gt_lines)
                correct += sum(1 for g, p in zip(gt_lines, pred_lines) if g == p)
    return round(correct / total * 100, 2) if total > 0 else None

# Hitung deteksi & akurasi
counts = count_detections(label_folder)
accuracy = compute_accuracy(label_folder, ground_truth_folder)

# Buat dataframe hasil
result = {
    'Model': 'yolov7',
    'Conf': 0.25,  # Ganti sesuai nilai deteksi kamu
    'IoU': 0.45,   # Ganti sesuai nilai deteksi kamu
    'Kerusuhan Massa': counts.get('kerusuhan_massa', 0),
    'Kerusuhan Api': counts.get('kerusuhan_api', 0),
    'Aman': counts.get('aman', 0),
    'Total Deteksi': sum(counts.values()),
    'Akurasi (%)': accuracy
}

df = pd.DataFrame([result])
print(df)

# Simpan ke CSV
df.to_csv("/content/hasil_perbandingan_MODEL_1_CONFT_025.csv", index=False)
print("Hasil disimpan ke: hasil_perbandingan_model.csv")


In [ ]:
import os
root_path = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder"
print(os.listdir(root_path))


##  Tampilkan Gambar Deteksi

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

result_path = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/Pantera-mosh-pit-at-Metallica-concert-in-Charlotte-NC_-May-31st-2025-1-_mp4-0029_jpg.rf.5cd5de73629576d55e50f0c4de7929d6.jpg'
img = Image.open(result_path)
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.title('Hasil Deteksi Kerusuhan')
plt.show()


## Deteksi Gambar Tunggal dengan YOLOv7

In [ ]:
# Path file yang ingin diuji (upload ke folder test_images/)
test_file = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/yolov7/kerusuhan-Konser-Musik-5/Test_Images/TEST1.jpeg'


# Deteksi
!python detect.py \
    --weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/yolov7/runs/train/exp/weights/best.pt \
    --conf 0.03 \
    --img-size 640 \
    --source {test_file} \
    --save-txt \
    --project /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/yolov7/results \
    --name deteksi_folder \
    --exist-ok

# ===== Tampilkan hasil gambar setelah deteksi =====
import os
from PIL import Image
import matplotlib.pyplot as plt

# Ambil nama file saja
filename = os.path.basename(test_file)

# Path hasil deteksi
result_image_path = f"/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/yolov7/results/deteksi_folder/{filename}"

# Tampilkan gambar
img = Image.open(result_image_path)
plt.figure(figsize=(10, 10))  # Perbesar tampilannya
plt.imshow(img)
plt.axis('off')
plt.title(f"Hasil Deteksi: {filename}")
plt.show()


# Evaluasi Model YOLOv7

Model pertama conf 0.2

In [ ]:
# Pindah ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7


#!wget -O utils/plots.py https://raw.githubusercontent.com/WongKinYiu/yolov7/main/utils/plots.py

# Evaluasi performa deteksi objek
!python test.py \
--weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt \
--data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
--conf-thres 0.2 \
--iou-thres 0.3 \
--save-json \
--save-txt


model pertama conf 0.25

In [ ]:
# Pindah ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7


#!wget -O utils/plots.py https://raw.githubusercontent.com/WongKinYiu/yolov7/main/utils/plots.py

# Evaluasi performa deteksi objek
!python test.py \
--weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp/weights/best.pt \
--data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
--conf-thres 0.25 \
--iou-thres 0.45 \
--save-json \
--save-txt


model ke dua conf 0.2

In [ ]:
# Pindah ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7


#!wget -O utils/plots.py https://raw.githubusercontent.com/WongKinYiu/yolov7/main/utils/plots.py

# Evaluasi performa deteksi objek
!python test.py \
--weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt \
--data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
--conf-thres 0.2 \
--iou-thres 0.3 \
--save-json \
--save-txt


model ke dua conf 0.25

In [ ]:
# Pindah ke direktori YOLOv7
%cd /content/drive/MyDrive/Skripsi_YOLOv7/yolov7


#!wget -O utils/plots.py https://raw.githubusercontent.com/WongKinYiu/yolov7/main/utils/plots.py

# Evaluasi performa deteksi objek
!python test.py \
--weights /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/runs/train/exp_continue/weights/best.pt \
--data /content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/data.yaml \
--conf-thres 0.25 \
--iou-thres 0.45 \
--save-json \
--save-txt


## Confusion Matrix dan Classification Report

In [ ]:
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Path folder ground truth dan prediksi YOLOv7 (format YOLO: class x_center y_center width height)
gt_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels"
pred_folder = "/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels"

# Nama kelas sesuai urutan ID
class_names = ['kerusuhan_massa', 'kerusuhan_api', 'aman']

y_true = []
y_pred = []

# Ambil nama semua file .txt di folder prediksi
for file_name in os.listdir(pred_folder):
    if file_name.endswith(".txt"):
        pred_file = os.path.join(pred_folder, file_name)
        gt_file = os.path.join(gt_folder, file_name)

        # Baca isi prediksi dan GT hanya jika file GT ada
        if os.path.exists(gt_file):
            with open(pred_file, 'r') as pf, open(gt_file, 'r') as gf:
                pred_lines = pf.readlines()
                gt_lines = gf.readlines()

                # Ambil semua ID class (kolom pertama)
                pred_ids = [int(line.split()[0]) for line in pred_lines]
                gt_ids = [int(line.split()[0]) for line in gt_lines]

                # Samakan panjang dengan asumsi satu objek per file (atau ambil minimum)
                for gt, pred in zip(gt_ids, pred_ids):
                    y_true.append(gt)
                    y_pred.append(pred)

# Validasi jumlah data
print(f"Total data dibandingkan: {len(y_true)}")

# Buat confusion matrix dan classification report
cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
report = classification_report(y_true, y_pred, labels=[0,1,2], target_names=class_names, zero_division=0)

# Tampilkan hasil
print("\n=== Classification Report ===\n")
print(report)

# Plot confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Ground Truth")
plt.title("Confusion Matrix")
plt.show()


## Akurasi Deteksi Kerusuhan Konser

In [ ]:
import os
from sklearn.metrics import accuracy_score

# PATH FOLDER LABEL
true_label_dir = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/kerusuhan-Konser-Musik-10/test/labels'
pred_label_dir = '/content/drive/MyDrive/Skripsi_YOLOv7/yolov7/results/deteksi_folder/labels'
#  EKSTRAK LABEL
y_true, y_pred = [], []

for file in sorted(os.listdir(true_label_dir)):
    if not file.endswith('.txt'):
        continue

    gt_path = os.path.join(true_label_dir, file)
    pred_path = os.path.join(pred_label_dir, file)

    # Ground truth
    with open(gt_path, 'r') as f:
        gt_lines = f.readlines()
    if not gt_lines:
        continue  # skip jika kosong
    gt_class = int(gt_lines[0].split()[0])
    y_true.append(gt_class)

    # Prediksi
    if os.path.exists(pred_path):
        with open(pred_path, 'r') as f:
            pred_lines = f.readlines()
        if pred_lines:
            pred_class = int(pred_lines[0].split()[0])
        else:
            pred_class = -1  # tidak ada prediksi
    else:
        pred_class = -1  # tidak ada file prediksi

    y_pred.append(pred_class)

# ======== HITUNG AKURASI =========
valid_pairs = [(t, p) for t, p in zip(y_true, y_pred) if p != -1]
if valid_pairs:
    y_true_valid, y_pred_valid = zip(*valid_pairs)
    accuracy = accuracy_score(y_true_valid, y_pred_valid)
    print(f"\nAkurasi Deteksi Keseluruhan: {accuracy * 100:.2f}%")
else:
    print("Tidak ada prediksi valid yang bisa dihitung akurasinya.")
